In [1]:
import pandas as pd
import numpy as np

In [2]:
# when True, uses part of the data, this is done to have faster testing times with initial production coding
PRODUCTION = False

In [3]:
data = pd.read_pickle("clustered.pkl")

In [4]:
if PRODUCTION is True:
    data = data.sample(frac=0.1)

In [5]:
dct = {"Longitude": "longitude", "Latitude": "latitude", "LSOA name": "LSOA", "Crime type": "crime_type", 
           "magp": "median_income", "avg age": "avg_age", "Borough": "borough", "Cluster": "cluster", "Season": "season"}
data.rename(columns=dct, inplace=True)

In [6]:
data.loc[data.cluster==0, "cluster"] = 13

In [7]:
data.drop(labels=["code", "LSOA code", "count"], axis=1, inplace=True)

In [8]:
# adding previous month to the data for an easier identifier 
data["prev_month"] = np.nan

lst_months = sorted(data.month.unique().tolist())
lst_prev = [12] + lst_months[:11]


i = 0
while i < 12:
    data.loc[data.month == lst_months [i], "prev_month"] = lst_prev[i]
    i += 1

data.prev_month = data.prev_month.astype(int)

In [9]:
# adding different types of crime statistics, currently the only one that is used is total monthly crime `monthly`
# this however easily could be build upon for different models/time series analyses given more time

# year
data["yearly"] = data.groupby(["year", "cluster"])["crime_type"].transform("count")
data["yearly_crime"] = data.groupby(["year", "crime_type", "cluster"])["crime_type"].transform("count")

# month
data["monthly"] = data.groupby(["year", "month", "cluster"])["crime_type"].transform("count")
data["monthly_crime"] = data.groupby(["year", "month", "crime_type", "cluster"])["crime_type"].transform("count")

# seasonality
data["seasonality"] = data.groupby(["year", "season", "cluster"])["crime_type"].transform("count")
data["seasonality_crime"] = data.groupby(["year", "season", "crime_type", "cluster"])["crime_type"].transform("count")

In [10]:
# we are going to add a time feature that has a rolling window, therefore we need to do some data wrangling steps
# due to the nature of the clustering, the data wrangling was a bit complicated and I did not find a way to do it
# more elegant then using 3 for loops (technically calling the function every time for a new cluster is also a for loop)
# however having the monthly data in seperate lists is usefull regardless to calculate the rolling mean more easily
# reminder, there are 13! clusters in total


lst_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
clusters = sorted(data.cluster.unique().tolist())
lst_months

def rolling_mean(data, list_years=lst_years, list_months=lst_months, size=4):
    """
    Outputs a list of the monthly crime per month sorted on time, this is used
    to then calculate a rolling mean for this particular cluster
    param: dataframe, this should only contain the cluster specific data
    list of years, list of months, rolling window size
    return: lst of the resulting rolling mean for this cluster
    """
    lst = []

    for i in list_years:
        for j in list_months:   
            lst.append(data.loc[(data.year==i) & (data.month==j)].monthly.unique()[0])
    

    windows = pd.Series(lst).rolling(size).mean()
    return windows.tolist()

In [11]:
# lists that represent the rolling means, just a number behind lst is used to indicated the cluster
# we can use previous months also in the test set as they will be known to the algorithm at the time of prediction

lst1 = rolling_mean(data.loc[data.cluster==1])
lst2 = rolling_mean(data.loc[data.cluster==2])
lst3 = rolling_mean(data.loc[data.cluster==3])
lst4 = rolling_mean(data.loc[data.cluster==4])
lst5 = rolling_mean(data.loc[data.cluster==5])
lst6 = rolling_mean(data.loc[data.cluster==6])
lst7 = rolling_mean(data.loc[data.cluster==7])
lst8 = rolling_mean(data.loc[data.cluster==8])
lst9 = rolling_mean(data.loc[data.cluster==9])
lst10 = rolling_mean(data.loc[data.cluster==10])
lst11 = rolling_mean(data.loc[data.cluster==11])
lst12 = rolling_mean(data.loc[data.cluster==12])
lst13 = rolling_mean(data.loc[data.cluster==13])

In [12]:
data[["rolling_months"]] = np.nan

In [13]:
data.loc[(data.month==1) &(data.year==2011) & (data.cluster==1), "rolling_months"] = lst1[0]

In [14]:
# here we again use a very bad tripple for loop, I was too strapped for time to figure out a more elegant/faster way
# it does run however quite fast due to the nature of serialized pandas objects

def rolling_mean_adder(data, lst , Z, list_years=lst_years, list_months=lst_months):
    """
    Adds the rolling mean values back into the dataframe
    param: dataframe of all the data, lst containing the cluster rolling mean
    list of years, list of months, rolling window size, Z an integer representing the cluster
    return: None
    """
    q = 0

    while q < len(lst):
        for i in list_years:
            for j in list_months:
                data.loc[(data.year==i) & (data.month==j) & (data.cluster==Z), "rolling_months"] = lst[q]
                q += 1

In [15]:
# lst number corresponding to cluster number to add data to the clusters

rolling_mean_adder(data, lst1, 1)
rolling_mean_adder(data, lst1, 2)
rolling_mean_adder(data, lst1, 3)
rolling_mean_adder(data, lst1, 4)
rolling_mean_adder(data, lst1, 5)
rolling_mean_adder(data, lst1, 6)
rolling_mean_adder(data, lst1, 7)
rolling_mean_adder(data, lst1, 8)
rolling_mean_adder(data, lst1, 9)
rolling_mean_adder(data, lst1, 10)
rolling_mean_adder(data, lst1, 11)
rolling_mean_adder(data, lst1, 12)
rolling_mean_adder(data, lst1, 13)

In [16]:
# add prev months crime rate this should be less effective then rolling window
data["prev_month_crime"] = np.nan

j = 0
while j < 12:
    data.loc[data.month == lst_months[j], "prev_month_crime"] = data.loc[data.prev_month == lst_prev[j]].monthly.unique()[0]
    j += 1

In [17]:
# timestamp was dropped in preprocessing, but for datatime ease we add it back in, all data was added on first day of month, so we can assing this
# this was done to try and train an ARIMA model, this model gave RAM errors asking for 300GB+ of RAM
data["day"] = 1

data["datetime"] = pd.to_datetime(data[["year", "month", "day"]])

In [18]:
# as data is timeseries we sort it one more time to be sure
data.sort_values("datetime", inplace=True)

In [19]:
# data.to_csv("final_data.csv")

In [20]:
# get rid of the incomplete years as they can't be used properly currently
data = data[~data.year.isin([2011, 2019])]

In [21]:
# split train and test based off whole years
train = data[~(data.year == 2018) & ~(data.year == 2017)]
test = data[(data.year == 2018) | (data.year == 2017)]

In [22]:
# Further Random Sampling to reduce train times
if PRODUCTION is True:
    train = train.sample(frac=0.1)

In [23]:
train.columns

Index(['reported_by', 'falls_withing', 'longitude', 'latitude', 'LSOA',
       'crime_type', 'year', 'month', 'borough', 'median_income', 'avg_age',
       'yearly', 'monthly', 'cluster', 'season', 'prev_month', 'yearly_crime',
       'monthly_crime', 'seasonality', 'seasonality_crime', 'rolling_months',
       'prev_month_crime', 'day', 'datetime'],
      dtype='object')

In [24]:
def seasonality_pick(data, season,season_crime):
    """
    dataframe gets transformed to represent the seasonality wanted
    """
    return data[["crime_type", "datetime", "borough", "median_income", "avg_age", "cluster", "year", "season", "month", "rolling_months", "prev_month_crime",season, season_crime]].copy()

In [25]:
# pick the training sets 
train_month = seasonality_pick(train, "monthly", "monthly_crime")
# train_year = seasonality_pick(train, "yearly", "yearly_crime")
# train_season = seasonality_pick(train, "seasonality", "seasonality_crime")

In [26]:
# pick the test sets 
test_month = seasonality_pick(test, "monthly", "monthly_crime")
# test_year = seasonality_pick(test, "yearly", "yearly_crime")
# test_season = seasonality_pick(test, "seasonality", "seasonality_crime")

In [27]:
# test_month

In [28]:
# train_month

In [29]:
# train_month.monthly

In [30]:
x_num = train_month.drop(columns=["crime_type", "borough", "cluster", "year", "season", "month", "datetime", "monthly"])
x_cat = train_month[["crime_type", "borough", "season"]]

In [31]:
x_cat.head()

,crime_type,borough,season
400204,Violent crime,Stockport,Winter
400210,Violent crime,Stockport,Winter
400211,Anti-social behaviour,Stockport,Winter
400209,Other theft,Stockport,Winter
400208,Burglary,Stockport,Winter


In [32]:
x_num.head()

,median_income,avg_age,rolling_months,prev_month_crime,monthly_crime
400204,27222,42.067611,2199.0,2264.0,262
400210,27222,45.860495,2199.0,2264.0,233
400211,27222,42.193424,2199.0,2264.0,711
400209,27222,45.860495,2199.0,2264.0,144
400208,27222,45.860495,2199.0,2264.0,197


In [33]:
train_month

,crime_type,datetime,borough,median_income,avg_age,cluster,year,season,month,rolling_months,prev_month_crime,monthly,monthly_crime
400204,Violent crime,2012-01-01,Stockport,27222,42.067611,1,2012,Winter,1,2199.00,2264.0,2091,262
400210,Violent crime,2012-01-01,Stockport,27222,45.860495,12,2012,Winter,1,2199.00,2264.0,1902,233
400211,Anti-social behaviour,2012-01-01,Stockport,27222,42.193424,12,2012,Winter,1,2199.00,2264.0,1902,711
400209,Other theft,2012-01-01,Stockport,27222,45.860495,12,2012,Winter,1,2199.00,2264.0,1902,144
400208,Burglary,2012-01-01,Stockport,27222,45.860495,12,2012,Winter,1,2199.00,2264.0,1902,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020805,Public order,2016-12-01,Rochdale,22880,41.383913,11,2016,Winter,12,2537.75,1895.0,2364,214
2020806,Vehicle crime,2016-12-01,Rochdale,22880,41.383913,11,2016,Winter,12,2537.75,1895.0,2364,165
2020807,Violence and sexual offences,2016-12-01,Rochdale,22880,41.383913,11,2016,Winter,12,2537.75,1895.0,2364,581
2020801,Criminal damage and arson,2016-12-01,Rochdale,22880,41.383913,11,2016,Winter,12,2537.75,1895.0,2364,269


## Encoding

In [34]:
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

# not using the Pipeline for now, as we don't want to scale the rolling_months and I don't have time to figure out how to pass this
# the Pipeline that is displayed is wrong and has to be build out with different Encoders regardless

# num_pipeline = Pipeline([
#     ("std_scaler", StandardScaler())
# ])

In [35]:
ord_enc = OrdinalEncoder()
borough_enc = LabelEncoder()
crime_enc = LabelEncoder()
inc_scl = StandardScaler()
age_scl = StandardScaler()

In [36]:
train_month.season = ord_enc.fit_transform(train_month[["season"]])
train_month.crime_type = crime_enc.fit_transform(train_month["crime_type"])
train_month.borough = borough_enc.fit_transform(train_month["borough"])
train_month.median_income = inc_scl.fit_transform(train_month[["median_income"]])
train_month.avg_age = age_scl.fit_transform(train_month[["avg_age"]])

In [37]:
test_month.season = ord_enc.transform(test_month[["season"]])
test_month.crime_type = crime_enc.transform(test_month["crime_type"])
test_month.borough = borough_enc.transform(test_month["borough"])
test_month.median_income = inc_scl.transform(test_month[["median_income"]])
test_month.avg_age = age_scl.transform(test_month[["avg_age"]])

In [38]:
test_month.head()

,crime_type,datetime,borough,median_income,avg_age,cluster,year,season,month,rolling_months,prev_month_crime,monthly,monthly_crime
2050143,0,2017-01-01,3,-0.901732,-1.059117,4,2017,3.0,1,2565.25,2264.0,2626,630
2050132,0,2017-01-01,3,-0.901732,0.984179,4,2017,3.0,1,2565.25,2264.0,2626,630
2050133,0,2017-01-01,3,-0.901732,0.984179,4,2017,3.0,1,2565.25,2264.0,2626,630
2050134,0,2017-01-01,3,-0.901732,0.984179,4,2017,3.0,1,2565.25,2264.0,2626,630
2050135,0,2017-01-01,3,-0.901732,0.984179,4,2017,3.0,1,2565.25,2264.0,2626,630


In [39]:
train_month.head()

,crime_type,datetime,borough,median_income,avg_age,cluster,year,season,month,rolling_months,prev_month_crime,monthly,monthly_crime
400204,15,2012-01-01,6,0.891052,1.110561,1,2012,3.0,1,2199.0,2264.0,2091,262
400210,15,2012-01-01,6,0.891052,1.894022,12,2012,3.0,1,2199.0,2264.0,1902,233
400211,0,2012-01-01,6,0.891052,1.136549,12,2012,3.0,1,2199.0,2264.0,1902,711
400209,6,2012-01-01,6,0.891052,1.894022,12,2012,3.0,1,2199.0,2264.0,1902,144
400208,2,2012-01-01,6,0.891052,1.894022,12,2012,3.0,1,2199.0,2264.0,1902,197


In [40]:
# train_month.season = ord_enc.inverse_transform(train_month[["season"]])
# train_month.crime_type = crime_enc.inverse_transform(train_month["crime_type"])
# train_month.borough = borough_enc.inverse_transform(train_month["borough"])
# train_month.median_income = inc_scl.inverse_transform(train_month[["median_income"]])
# train_month.avg_age = age_scl.inverse_transform(train_month[["avg_age"]])

In [41]:
# train_month.head(10)

In [42]:
# num_attr = list(x_num)
# cat_attr = list(x_cat)

# full_pipeline = ColumnTransformer([
#     ("num",  num_pipeline, num_attr),
#     ("cat", OneHotEncoder(sparse=False), cat_attr),
# ])

## Models
Here we have 3 different models, counting GradientBoosting and xGDboost as same type of models, that can be used to determine the crime outcome of a cluster for the next month(s), with varying results. All models could work with new data, however from a time-series perspective opting to use a continous training model would have been better. Current models do not satisfy the criteria to be considered standard practice Time-Series models, therefore one should first look into implenting a more appropriate time-series model before continous training is applied.

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [106]:
X_train = train_month[["crime_type", "borough", "median_income", "avg_age", "season", "month", "rolling_months"]]
# X_test = test_clst1[["crime_type", "borough", "median_income", "avg_age", "season", "month", "rolling_months"]]
y_train = train_month["monthly"]
# y_test = test_clst1["monthly"]

In [76]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def time_metrics(y_test, y_pred, model):
    """
    print the time metrics
    """
    print(f"Error metrics for the {model} prediction model")
    print("\n")
    print('Mean Absolute Error:', round(mean_absolute_error(y_test, y_pred), 3))
    print('Mean Squared Error:', round(mean_squared_error(y_test, y_pred), 3))
    print('Root Mean Squared Error:', round(np.sqrt(mean_squared_error(y_test, y_pred)), 3))
    print('R2 score:', round(r2_score(y_test, y_pred), 3))


In [86]:
# # we filter for the clusters
# train_clst1 = train_month.loc[train_month.cluster==1]
# test_clst1 = test_month.loc[test_month.cluster==1]

In [107]:
def test_model(data, model, model_name):
    """
    Tests the outcomes of the trained model per cluster
    param: general test set, the variable storing the model function, model name as a string
    return: prints of the error metrics
    """
    for i in clusters:
        test = data.loc[data.cluster==i]
        X_test = test[["crime_type", "borough", "median_income", "avg_age", "season", "month", "rolling_months"]]
        y_test = test["monthly"]
        y_pred = model.predict(X_test)
        print("\n")
        print(f"Cluster {i}")
        time_metrics(y_test, y_pred, model_name)

In [110]:
# test_model(test_month, lin_reg, "Linear")

### Linear Model

In [94]:
# X_train.cluster

In [108]:
lin_reg = LinearRegression();
lin_reg.fit(X_train, y_train);

In [81]:
y_pred = lin_reg.predict(X_test)

In [111]:
test_model(test_month, lin_reg, "Linear")



Cluster 1
Error metrics for the Linear prediction model


Mean Absolute Error: 118.476
Mean Squared Error: 22557.2
Root Mean Squared Error: 150.191
R2 score: 0.601


Cluster 2
Error metrics for the Linear prediction model


Mean Absolute Error: 183.241
Mean Squared Error: 51811.182
Root Mean Squared Error: 227.621
R2 score: -0.368


Cluster 3
Error metrics for the Linear prediction model


Mean Absolute Error: 396.493
Mean Squared Error: 184967.325
Root Mean Squared Error: 430.078
R2 score: -6.71


Cluster 4
Error metrics for the Linear prediction model


Mean Absolute Error: 177.758
Mean Squared Error: 49483.475
Root Mean Squared Error: 222.449
R2 score: -0.582


Cluster 5
Error metrics for the Linear prediction model


Mean Absolute Error: 581.706
Mean Squared Error: 360756.919
Root Mean Squared Error: 600.63
R2 score: -8.188


Cluster 6
Error metrics for the Linear prediction model


Mean Absolute Error: 229.14
Mean Squared Error: 87063.98
Root Mean Squared Error: 295.066
R2 score

### Random Forest

In [112]:
rdf_reg = RandomForestRegressor()
rdf_reg.fit(X_train, y_train);

In [113]:
# rdf_pred = rdf_reg.predict(X_test)

In [114]:
test_model(test_month, rdf_reg, "Random Forest")



Cluster 1
Error metrics for the Random Forest prediction model


Mean Absolute Error: 175.054
Mean Squared Error: 47201.447
Root Mean Squared Error: 217.259
R2 score: 0.166


Cluster 2
Error metrics for the Random Forest prediction model


Mean Absolute Error: 224.233
Mean Squared Error: 73085.568
Root Mean Squared Error: 270.343
R2 score: -0.929


Cluster 3
Error metrics for the Random Forest prediction model


Mean Absolute Error: 194.929
Mean Squared Error: 69737.955
Root Mean Squared Error: 264.079
R2 score: -1.907


Cluster 4
Error metrics for the Random Forest prediction model


Mean Absolute Error: 209.732
Mean Squared Error: 84329.231
Root Mean Squared Error: 290.395
R2 score: -1.696


Cluster 5
Error metrics for the Random Forest prediction model


Mean Absolute Error: 214.109
Mean Squared Error: 71331.777
Root Mean Squared Error: 267.08
R2 score: -0.817


Cluster 6
Error metrics for the Random Forest prediction model


Mean Absolute Error: 226.677
Mean Squared Error: 100666

### GradiantBoost

In [115]:
xgb_reg = GradientBoostingRegressor()
xgb_reg.fit(X_train, y_train)

GradientBoostingRegressor()

In [116]:
# xgb_pred = xgb_reg.predict(X_test)

In [118]:
test_model(test_month, xgb_reg, "Gradiant Boosting")



Cluster 1
Error metrics for the Gradiant Boosting prediction model


Mean Absolute Error: 162.683
Mean Squared Error: 43139.74
Root Mean Squared Error: 207.701
R2 score: 0.237


Cluster 2
Error metrics for the Gradiant Boosting prediction model


Mean Absolute Error: 181.159
Mean Squared Error: 45914.749
Root Mean Squared Error: 214.277
R2 score: -0.212


Cluster 3
Error metrics for the Gradiant Boosting prediction model


Mean Absolute Error: 201.974
Mean Squared Error: 59935.453
Root Mean Squared Error: 244.817
R2 score: -1.498


Cluster 4
Error metrics for the Gradiant Boosting prediction model


Mean Absolute Error: 178.712
Mean Squared Error: 44945.837
Root Mean Squared Error: 212.004
R2 score: -0.437


Cluster 5
Error metrics for the Gradiant Boosting prediction model


Mean Absolute Error: 181.225
Mean Squared Error: 53330.59
Root Mean Squared Error: 230.934
R2 score: -0.358


Cluster 6
Error metrics for the Gradiant Boosting prediction model


Mean Absolute Error: 238.3
Mean 

### xGdBoost

In [119]:
import xgboost as xgb

In [120]:
xtr_bst = xgb.XGBRFRegressor();
xtr_bst.fit(X_train, y_train);

C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [121]:
# xtr_bst_pred = xtr_bst.predict(X_test)

In [122]:
# time_metrics(y_test, xtr_bst_pred, "Extreme Gradient Boosting")
test_model(test_month, xtr_bst, "Extreme Gradient Boosting")

C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future ver



Cluster 1
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 169.778
Mean Squared Error: 45892.741
Root Mean Squared Error: 214.226
R2 score: 0.189


Cluster 2
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 173.901
Mean Squared Error: 43751.42
Root Mean Squared Error: 209.168
R2 score: -0.155


Cluster 3
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 201.696
Mean Squared Error: 60534.278
Root Mean Squared Error: 246.037
R2 score: -1.523


Cluster 4
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 184.964
Mean Squared Error: 50071.722
Root Mean Squared Error: 223.767
R2 score: -0.601


C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future ver



Cluster 5
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 183.975
Mean Squared Error: 53840.205
Root Mean Squared Error: 232.035
R2 score: -0.371


Cluster 6
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 251.123
Mean Squared Error: 102506.745
Root Mean Squared Error: 320.167
R2 score: -3.772


Cluster 7
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 329.844
Mean Squared Error: 148901.902
Root Mean Squared Error: 385.878
R2 score: -3.216


Cluster 8
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 314.168
Mean Squared Error: 137156.302
Root Mean Squared Error: 370.346
R2 score: -1.145


Cluster 9
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 191.479
Mean Squared Error: 49028.559
Root Mean Squared Error: 221.424
R2 score: 0.077


Cluster 10
Error metrics for the Extreme Gradient Boosti

C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future ver



Cluster 12
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 140.462
Mean Squared Error: 27160.014
Root Mean Squared Error: 164.803
R2 score: 0.115


Cluster 13
Error metrics for the Extreme Gradient Boosting prediction model


Mean Absolute Error: 148.733
Mean Squared Error: 33816.09
Root Mean Squared Error: 183.892
R2 score: -0.254


C:\Users\Jarl\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
